# Extracting ROMs from ALEXEC.LDA

There are ROMs in them there hills. Here we extract the ROMs from ALEXEC.LDA to find what version of Tempest is in there.

In [48]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

The file `ALEXEC.LDA` is in `LDA` format(!). This is chunks of data with a header and checksum.

For example (all words are little-endian):
```
Sentinel: 0x0100
Length: 0x0B00
Load Address: 0x0090
Data: 0x0102030405
Checksum: 0x09
```

You keep reading the chunks above until there are none left! 

See http://www.bitsavers.org/pdf/dec/pdp11/rt11/v5.6_Aug91/AA-PD6PA-TC_RT-11_Volume_and_File_Formats_Manual_Aug91.pdf

In [73]:
word = lambda x: (x).to_bytes(2,"little")
read_word = lambda x: int.from_bytes(x,"little")

BLOCK_HEADER = b'\x01\x00'
LDA_END = 0x0000

al = open("tempest/ALEXEC.LDA",'rb')
blocks = []
while True:
    sentinel = al.read(2)
    if sentinel != BLOCK_HEADER:
        raise Exception("Block Header Expected")
    
    len_data = read_word(al.read(2)) - 6
    if len_data == LDA_END:
        break
    addr = read_word(al.read(2))
    data = al.read(len_data)
    checksum = al.read(1)
    blocks += [(hex(addr),addr,len(data),data)]
        
output = b'\x00' * 0xFFFF
output_bytes = bytearray(output)
for hex_addr,addr,len_data,data in blocks:
    output_bytes[addr:addr+len_data] = data


## Running Tempest 'Version: 2'

In [74]:

"""
Project #: 28903	Project Programmer: Dave Theurer	Project Leader: Morgan Hoff

Documentation Disk(s): 36A,36B,36C	Project Name: Tempest	Date: 11-25-81	Version: 2


Verification control filename: 002X2.DAT	Verification file: ALEXEC.LDA


Part #	PCB Loc.	Description		Start Addr	Generic name	Part Size	Bits Used
------	--------	-----------		----------	------------	---------	---------

136002-123	N3	Vect.Gen.Erom		3000		2716		2Kx8		0-7
136002-124	R3	Vect.Gen.Erom		3800		2716		2Kx8		0-7

136002-113	D1	Program Erom		9000		2716		2Kx8		0-7
136002-114	E1	Program Erom		9800		2716		2Kx8		0-7
136002-115	F1	Program Erom		A000		2716		2Kx8		0-7
136002-216	H1	Program Erom		A800		2716		2Kx8		0-7
136002-217	J1	Program Erom		B000		2716		2Kx8		0-7
136002-118	K1	Program Erom		B800		2716		2Kx8		0-7
136002-119	L/M1	Program Erom		C000		2716		2Kx8		0-7
136002-120	N1	Program Erom		C800		2716		2Kx8		0-7
136002-121	P1	Program Erom		D000		2716		2Kx8		0-7
136002-222	R1	Program Erom		D800		2716		2Kx8		0-7
"""
OUT_DIR="ALEXEC.LDA Roms/tempest2/"
open(f"{OUT_DIR}136002-123.np3",'wb').write(bytes(output_bytes[0x3000:0x3800]))
open(f"{OUT_DIR}136002-124.r3",'wb').write(bytes(output_bytes[0x3800:0x4000]))

open(f"{OUT_DIR}136002-113.d1",'wb').write(bytes(output_bytes[0x9000:0x9800]))
open(f"{OUT_DIR}136002-114.e1",'wb').write(bytes(output_bytes[0x9800:0xA000]))
open(f"{OUT_DIR}136002-115.f1",'wb').write(bytes(output_bytes[0xA000:0xA800]))

open(f"{OUT_DIR}136002-216.h1",'wb').write(bytes(output_bytes[0xA800:0xB000]))
open(f"{OUT_DIR}136002-116.h1",'wb').write(bytes(output_bytes[0xA800:0xB000]))

open(f"{OUT_DIR}136002-217.j1",'wb').write(bytes(output_bytes[0xB000:0xB800]))
open(f"{OUT_DIR}136002-118.k1",'wb').write(bytes(output_bytes[0xB800:0xC000]))
open(f"{OUT_DIR}136002-119.lm1",'wb').write(bytes(output_bytes[0xC000:0xC800]))
open(f"{OUT_DIR}136002-120.mn1",'wb').write(bytes(output_bytes[0xC800:0xD000]))
open(f"{OUT_DIR}136002-121.p1",'wb').write(bytes(output_bytes[0xD000:0xD800]))
open(f"{OUT_DIR}136002-222.r1",'wb').write(bytes(output_bytes[0xD800:0xE000]))


2048

In [75]:
!mame -window -rompath /home/robert/Dev/tempest/notebooks/ALEXEC.LDA\ Roms/ tempest2

Unable to find the BGFX path bgfx, please install it or fix the bgfx_path setting to use the BGFX renderer.
136002-116.h1 WRONG CHECKSUMS:
    EXPECTED: CRC(7356896c) SHA1(a013ede292189a8f5a907de882ee1a573d784b3c)
       FOUND: CRC(aeb0f7e9) SHA1(a5cc25015b98692673cfc1c7c2e9634efd750870)
Average speed: 100.00% (24 seconds)


It works, however there are a couple of bytes that are different. It looks like we have a previously unknown ROM file in 136002-216.h1. (Note I've renamed it 136002-116.h1 just to get it to run with warnings above.)

In [69]:
!diff -y -W 150 <(dd if=ALEXEC.LDA\ Roms/tempest2/136002-116.h1|xxd) <(dd if=roms/136002-116.h1|xxd) |grep '  |	'

4+0 records in
4+0 records out
4+0 records in
4+0 records out
2048 bytes (2.0 kB, 2.0 KiB) copied, 1.9455e-05 s, 105 MB/s
2048 bytes (2.0 kB, 2.0 KiB) copied, 2.3515e-05 s, 87.1 MB/s
000000a0: 8d25 0160 b98a 0229 fc99 8a02 4c98 a3e1  .%.`...)....L...	  |	000000a0: 8d25 0160 b98a 0229 fc99 8a02 4c98 a3b2  .%.`...)....L...
00000260: 69aa a930 a200 2017 ab20 92aa 4ce7 a820  i..0.. .. ..L.. 	  |	00000260: 69aa a930 a200 2017 ab20 92aa 4cb4 a820  i..0.. .. ..L.. 


Stop press. It looks like 215 is the same as 316:

In [71]:
!diff -y -W 150 <(dd if=ALEXEC.LDA\ Roms/tempest2/136002-116.h1|xxd) <(dd if=roms/136002-316.h1|xxd) |grep '  |	'

4+0 records in
4+0 records out
2048 bytes (2.0 kB, 2.0 KiB) copied, 1.8446e-05 s, 111 MB/s
4+0 records in
4+0 records out
2048 bytes (2.0 kB, 2.0 KiB) copied, 1.7627e-05 s, 116 MB/s


## Running Tempest 2A(Alt)

In [76]:
# Tempest '2A'
"""
TEMPST.DOC:
Project #:  28903			Project Programmer:  Dave Theurer		Project Leader:  Morgan Hoff

Documentation Disk(s): 36A,36B,36C		Project Name:  Tempest			Date:  12-17-81		Version: 2A (alt)


Verification control filename: 002X2.DAT	Verification file: ALEXEC.LDA


Part #		PCB Loc.	Description		Start Addr	Generic name	Part Size	Bits Used
------		--------	-----------		----------	------------	---------	---------	

136002-138	N/P3		VG EROM			3000		2532		4KX8		7->0				

136002-237	P1		PROGRAM EROM		9000		2532		4KX8		7->0
136002-136	L/M1		PROGRAM EROM		A000		2532		4KX8		7->0
136002-235	J1		PROGRAM EROM		B000		2532		4KX8		7->0
136002-134	F1		PROGRAM EROM		C000		2532		4KX8		7->0
136002-133	D1		PROGRAM EROM		D000		2532		4KX8		7->0
				(ind. self test)
"""
OUT_DIR="ALEXEC.LDA Roms/tempest/"

open(f"{OUT_DIR}136002-123.np3",'wb').write(bytes(output_bytes[0x3000:0x3800]))
open(f"{OUT_DIR}136002-124.r3",'wb').write(bytes(output_bytes[0x3800:0x4000]))

open(f"{OUT_DIR}136002-138.np3",'wb').write(bytes(output_bytes[0x3000:0x4000]))

# Notice we have the addresses in reverse order, e.g. 136002-237 is written to 0xD000 rather
# than 0x9000.
open(f"{OUT_DIR}136002-237.p1",'wb').write(bytes(output_bytes[0xD000:0xE000]))
open(f"{OUT_DIR}136002-136.lm1",'wb').write(bytes(output_bytes[0xC000:0xD000]))
open(f"{OUT_DIR}136002-235.j1",'wb').write(bytes(output_bytes[0xB000:0xC000]))
open(f"{OUT_DIR}136002-134.f1",'wb').write(bytes(output_bytes[0xA000:0xB000]))
open(f"{OUT_DIR}136002-133.d1",'wb').write(bytes(output_bytes[0x9000:0xA000]))


4096

This runs cleanly and without warnings so the binaries here are for what mame refers to as `rev 3`.

In [77]:
!mame -window -rompath /home/robert/Dev/tempest/notebooks/ALEXEC.LDA\ Roms/ tempest

Unable to find the BGFX path bgfx, please install it or fix the bgfx_path setting to use the BGFX renderer.
Average speed: 100.01% (2 seconds)


This means that our 'new' ROM file is v3 in the old 'v2' format (i.e. 2K ROMs rather than 4K ROMs).